# OND Case 3

Mass point insertion.

First we import

In [1]:
import numpy as np
import random
import uuid
import timeit

# Definitions: classes

In [2]:
class Car:
    def __init__(self):
        self.id = str(uuid.uuid1())
    
    def __str__(self):
        return self.id
    
class Location:
    def __init__(self, car):
        self.id = str(uuid.uuid1())
        self.lat = random.random()
        self.lng = random.random()
        self.car = car
        
    def __str__(self):
        return "{2}: {0} by {1}".format(self.lat, self.lng, self.car)

# Generate the data

In [3]:
cars = [Car() for i in range(0, 100)]
cars

In [4]:
locations = [Location(random.choice(cars)) for i in range(0, 10_000)]
locations[0]

# Graph Database

In [5]:
from py2neo import Graph, Path, Node, Relationship, NodeMatcher
graph = Graph()
matcher = NodeMatcher(graph)

# clear db
graph.evaluate("MATCH (n) DETACH DELETE n")

In [6]:
tx = graph.begin()
for car in cars:
    tx.create(Node("Car", id=car.id))
tx.process()
tx.commit()

In [7]:
def neo4j_create():
    tx = graph.begin()
    for location in locations:
        loc = Node("Location", lat=location.lat, lng=location.lng)
        car = matcher.match('Car', id=location.car.id).first()
        tx.create(loc)
        tx.create(car)
        rel = Relationship(car, "WAS_AT", loc)
        tx.create(rel)
    tx.commit()

timeit.timeit(stmt=neo4j_create, number=1)

71.55504949299211

# OpenTSDB

In [8]:
from opentsdb import TSDBClient

tsdb = TSDBClient(host='localhost')
tsdb.is_alive()

True

In [9]:
def opentsdb_create():
    for location in locations:
        tsdb.send('logging.coordinates.lat', location.lat, car=location.car.id)
        tsdb.send('logging.coordinates.lng', location.lng, car=location.car.id)
        
    tsdb.close()
    tsdb.wait()
timeit.timeit(stmt=opentsdb_create, number=1)

Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue i

Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue i

Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue i

Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue i

Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue i

Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue is full.
Drop oldest metric because Queue i

4.764431028001127

# MongoDB

In [10]:
from pymongo import MongoClient

mongo = MongoClient()
mongodb = mongo['test-db']
mongocol = mongodb['locations']

In [11]:
mongocol.drop()
def mongo_write():
    mongocol.insert_many([{"lat": location.lat,"lng": location.lng,"car": {"identifier": location.car.id}}for location in locations])
    
timeit.timeit(stmt=mongo_write, number=1)

0.2714383889979217